# Elasticsearch

In this exercise, you'll first build an Elasticsearch index of a toy document collection, then request various term statistics from that index.

Before starting, make sure that you've 

- 1) Downloaded and started Elasticsearch
- 2) Installed the `elasticsearch` Python package (`conda install elasticsearch`)

See [this document](https://github.com/kbalog/ir-course/tree/master/code/elasticsearch) for detailed instructions.

In [1]:
from elasticsearch import Elasticsearch

import ipytest
import pytest

ipytest.autoconfig()

This is to check that the Elasticsearch service is running on your machine.

In [2]:
es = Elasticsearch()

## Indexing

We use a toy data collection with 5 documents, each with title and content fields.

In [3]:
DOCS = [
    {'doc_id': 'D1',
     'title': 'First document',
     'content': 'House on the hill'
    },
    {'doc_id': 'D2',
     'title': 'Second title',
     'content': 'Downtown Stavanger is beautiful'
    },
    {'doc_id': 'D3',
     'title': 'First, second, and third',
     'content': 'Never step on snakes'
    },
    {'doc_id': 'D4',
     'title': 'Document number four',
     'content': 'House, house. It\'s a beautiful house you have'
    },
    {'doc_id': 'D5',
     'title': 'This document is the last document',
     'content': 'There can be only one matching result'
    }    
]

In [4]:
INDEX_SETTINGS = {  # single shard with a single replica
    'settings' : {
        'index' : {
            'number_of_shards' : 1,
            'number_of_replicas' : 1
        }
    }
}

In [5]:
INDEX_NAME = 'test5'

In [6]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test5'}

Add documents in `DOC` to the index.

In [7]:
for doc in DOCS:
    es.index(index=INDEX_NAME, id=doc['doc_id'], body={'title': doc['title'], 'content': doc['content']})

## Term statistics

Complete the methods below for getting various term statistics from the index.

Consult [this notebook](https://github.com/kbalog/ir-course/blob/master/code/elasticsearch/Elasticsearch.ipynb) for the interpretation of term vector statistics.

In [8]:
def get_doc_term_freqs(index_name, doc_id, field):
    """Returns the terms along with their frequencies contained in a given document.
    
    Params:
        index_name: Name of index
        doc_id: Document ID
        field: Field name
    
    Returns:
        Dict with terms as keys and corresponding frequencies (i.e., 
        number of occurrences within the given document field) as values.
    """
    tv = es.termvectors(index=index_name, id=doc_id, fields=field, term_statistics=True)
    if tv['_id'] != doc_id:
        return None
    if field not in tv['term_vectors']:
        return None
    term_freqs = {}
    for term, term_stat in tv['term_vectors'][field]['terms'].items():
        term_freqs[term] = term_stat['term_freq']
    return term_freqs

In [9]:
def get_doc_field_len(index_name, doc_id, field):
    """Returns the length of a given document field.
    
    Length is defined as the total number of terms contained in that field.
    
    Params:
        index_name: Name of index
        doc_id: Document ID
        field: Field name
    
    Returns:
        Field length (int).    
    """
    term_freqs = get_doc_term_freqs(index_name, doc_id, field)
    if term_freqs is not None:
        return sum(term_freqs.values())
    return None

In [10]:
def get_doc_containing_term(index_name, field, term):
    """Returns any document ID that contains term in a given field or None.
    
    Params:
        index_name: Name of index
        field: Field name
        term: Term

    Return:
        ID of a document that contains `term` or None.
    """
    # Use a boolean query to find a document that contains the term.
    hits = es.search(index=index_name, body={'query': {'match': {field: term}}},
                               _source=False, size=1).get('hits', {}).get('hits', {})
    return hits[0]['_id'] if len(hits) > 0 else None

In [11]:
def get_term_doc_count(index_name, field, term):
    """Returns the total number of documents that contain a given term within a specific field.
    
    Params:
        index_name: Name of index
        field: Field name
        term: Term
        
    Return:
        Number of documents (int) that contain the given term within `field`.
    """
    # Find a document that contains the term.
    doc_id = get_doc_containing_term(index_name, field, term)
    if doc_id is None:
        return 0
    # Request term statistics for that document and extract the 
    # requested information from there.
    tv = es.termvectors(index=index_name, id=doc_id, fields=field, term_statistics=True)
    return tv['term_vectors'][field]['terms'][term]['doc_freq']

In [12]:
def get_term_coll_freq(index_name, field, term):
    """Returns the total collection term frequency of a term in a given field.
    
    Params:
        index_name: Name of index
        field: Field name
        term: Term
        
    Return:
        Total number of occurrences of `term` (int) in all documents within `field`.
    """
    # Find a document that contains the term.
    doc_id = get_doc_containing_term(index_name, field, term)
    if doc_id is None:
        return 0
    # Request term statistics for that document and extract the 
    # requested information from there.
    tv = es.termvectors(index=index_name, id=doc_id, fields=field, term_statistics=True)
    return tv['term_vectors'][field]['terms'][term]['ttf']

Tests.

In [14]:
%%run_pytest[clean]

def test_doc_term_freqs():
    assert get_doc_term_freqs(INDEX_NAME, 'D2', 'title') == {'second': 1, 'title': 1}
    assert get_doc_term_freqs(INDEX_NAME, 'D4', 'content') == {'a': 1, 'beautiful': 1, 'have': 1,
                                                               'house': 3, 'it\'s': 1, 'you': 1}
    
def test_doc_field_len():
    assert get_doc_field_len(INDEX_NAME, 'D2', 'title') == 2
    assert get_doc_field_len(INDEX_NAME, 'D4', 'content') == 8
    
def test_doc_containing_term():
    assert get_doc_containing_term(INDEX_NAME, 'title', 'document') in ['D1', 'D4', 'D5']
    assert get_doc_containing_term(INDEX_NAME, 'content', 'house') in ['D1', 'D4']
    
def test_term_doc_count():
    assert get_term_doc_count(INDEX_NAME, 'title', 'document') == 3
    assert get_term_doc_count(INDEX_NAME, 'content', 'house') == 2    
    
def test_term_coll_freq():
    assert get_term_coll_freq(INDEX_NAME, 'title', 'this') == 1
    assert get_term_coll_freq(INDEX_NAME, 'title', 'document') == 4
    assert get_term_coll_freq(INDEX_NAME, 'content', 'house') == 4       

.....                                                                              [100%]
5 passed in 0.06s


## Feedback

Please give (anonymous) feedback on this exercise by filling out [this form](https://forms.gle/2jPayczbFhEcC9K68).